In [27]:
import numpy as np
from numpy.core.defchararray import translate

In [1]:
import src.triangle_ray_intersection as triangle_ray_intersection

In [ ]:
names = ['thorax', 'neck', 'a_L1', 'a_L2', 'a_R1', 'a_R2', 'leg_f_L1', 'leg_f_L2', 'leg_f_R1', 'leg_f_R2', 'leg_m_L1', 'leg_m_L2', 'leg_m_R1', 'leg_m_R2', 'a_R0', 'a_L0', 'leg_f_R0', 'leg_f_L0', 'leg_m_R0', 'leg_m_L0', 'eye_L', 'eye_R', 'm_L0', 'm_R0', 'm_L1', 'm_R1']

In [2]:
t = triangle_ray_intersection.CollisionDetector(obj_folder="./example_obj_folder/", skeleton_json_path="", pose_csv="ant_3d_validrows.csv")

'seed' generated an exception: Folder ./example_obj_folder/seed ignored


True


In [3]:
t.obj.get_obj_trimesh(1964)

KeyError: 1964

In [ ]:
for i in range(2400):
    try:
        location, norm = t.get_collisions(i)
    except:
        location, norm = [], []
    if len(location) > 0:
        print(i)

In [ ]:

scene = t.visualise_collision(1752)

scene.show(viewer="gl")

In [ ]:
scene = t.animal.visualise_animal(1064)
scene.show(viewer="gl", line_settings={'point_size':10})

In [4]:
p = t.animal.get_animal_pose(1064)

[min(t.animal._pose_dict.keys()), max(t.animal._pose_dict.keys())]

[531, 1980]

In [3]:
V = triangle_ray_intersection.Viewer(t.animal, 1064)

[  7.77908316  -0.93858295 213.04610846]
[  7.02242681  -2.10629983 211.6460899 ]
[  5.95357031  -1.59772389 211.49443743]
[  5.14443406  -3.15192016 211.37856096]
[  6.59608275  -1.60907317 211.08816841]
[  7.02375957  -1.71159104 210.54070367]
[  6.77599078  -1.27349499 212.22641301]
[  7.61572139  -1.17526076 212.5672636 ]
[  7.55159642   1.08769848 215.86509138]
[  7.8259742   -1.05910208 212.62304382]
[  7.51887914  -1.24686112 212.92233106]
[  5.96610119  -1.56329485 210.78405983]
[  6.34868835  -2.14999994 210.58981327]
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648

In [7]:
V.scene_widget1.scene.update()

AttributeError: 'Scene' object has no attribute 'update'